In [ ]:
import os
from pathlib import Path


cwd = os.getcwd()
display(cwd)
path_parent = Path(os.path.dirname(os.path.dirname(os.getcwd())))
test = Path(cwd)
display(test)
test2 = test.parent.parent
display(test2)

files = path_parent / "Training_Dataset" / "metadata"
#display(files)

In [1]:
#Extracting Features for every audio file

import librosa
import librosa.display
from scipy.io import wavfile as wav
import IPython.display as ipd
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Creating a function that extracts the MFCC features of an audio file
def extract_features(file_name):
    
    try:
        
        # Librosa extraction of audio array and sampling rate
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') # resampling at a "faster rate as opposed to higher quality"
        # MFCC feature extraction of audio - mfccs is mfcc sequence (array), n_mfcc is number of MFCCs to return
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        
        # why ????
        mfccsscaled = np.mean(mfccs.T, axis=0) # transpose and get mean of the samples
        
    except Exception as e:
        print("Error encountered while parsing file ", file_name)
        return None
    
    return mfccsscaled
    #return mfccs

#### Extracting features from all audio signals using the extract_features funnction. 

In [2]:
# Set the path to the full UrbanSound dataset

# Set the path to the full UrbanSound dataset
root_path = Path(os.getcwd()).parent.parent # Software Folder
fulldatasetpath = root_path / "Training_Dataset" / "audio"
metadata = pd.read_csv(root_path / "Training_Dataset" / "metadata" / "UrbanSound8k.csv")

categories = ['dog_bark', 'car_horn', 'gun_shot', 'siren']

features = []

for index, row in metadata.iterrows():
    
    # Extract filename and category
    category_str = row["class_name"]
    
    # Loop through metadata comparing the categories
    if category_str in categories:
        file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
        file_name = Path(file_name) #Convert to pathlib object for OS compatibility
        # Extract features for each wave file
        data = extract_features(file_name)
        features.append([data, category_str])
    else:
        continue

# Convert into a Panda dataframee
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ',len(featuresdf), 'files')
display(featuresdf)



Finished feature extraction from  2732 files


,feature,class_label
0,"[-215.79301, 71.66612, -131.81377, -52.09133, ...",dog_bark
1,"[-196.18527, 114.94506, -14.661185, 1.2298629,...",car_horn
2,"[-202.99025, 111.43653, -28.663647, 14.733859,...",car_horn
3,"[-206.01172, 90.92227, -25.063505, 29.089556, ...",car_horn
4,"[-204.60739, 103.6529, -29.975662, 27.412035, ...",car_horn
...,...,...
2727,"[-399.2257, 136.81903, -51.964222, 37.02399, -...",car_horn
2728,"[-346.72733, 87.48847, -46.265022, 52.74884, -...",car_horn
2729,"[-304.61316, 112.6199, -47.161945, 37.00349, -...",car_horn
2730,"[-344.71423, 126.75813, -56.17717, 36.070927, ...",car_horn


In [3]:
# Normalize the dataframe such that there are the same number of files per class_label
# This ensures that no one category has an advantage when the model is being trained

print(featuresdf.class_label.count()) # 2732

# Create dictionary of dataframes
frames = {}
categories = ['dog_bark', 'car_horn', 'gun_shot', 'siren']

arr_Size = []

for label in categories:
    frames[label] = featuresdf[featuresdf['class_label'] == label]
    # Extract shape and get number of rows
    rNc = frames[label].shape
    # Gets number of rows
    arr_Size.append(rNc[0])
    print(label, rNc[0])

# Take the minimum size from size array
minSize = min(arr_Size)

# Utilize minimum size to slice rows such that only the minimum size is maintained
for label in frames:
    frames[label] = frames[label].sample(minSize)
    print(frames[label].shape[0])
    
# Concatenate all dataframes in dictionary of dataframes
# Place the concatenated frame back in featuresdf
# Reindex
result = pd.concat(frames)
#display(type(result))
features_temp = pd.DataFrame()
features_temp = result[["feature", "class_label"]]

# Reindex features_temp
features_temp = features_temp.reset_index(drop=True)
display(features_temp)

2732
dog_bark 1000
car_horn 429
gun_shot 374
siren 929
374
374
374
374


,feature,class_label
0,"[-314.87665, 166.32927, 4.413409, 16.052296, 0...",dog_bark
1,"[-181.81837, 137.67917, -77.676834, -45.46656,...",dog_bark
2,"[-323.49872, 140.18324, -11.821219, 14.91496, ...",dog_bark
3,"[-585.21686, 93.08137, -30.279652, 6.409308, -...",dog_bark
4,"[-224.30125, 173.66426, -23.907955, 18.138966,...",dog_bark
...,...,...
1491,"[-255.47444, 148.7255, -50.32463, 5.921347, -1...",siren
1492,"[-214.6939, 82.65424, -34.92253, 1.847175, -28...",siren
1493,"[-271.99802, 143.69476, -77.24437, -0.6722971,...",siren
1494,"[-422.82904, 160.23573, -64.7399, -24.466932, ...",siren


In [4]:
# Send temp features to features df
featuresdf = features_temp
display(featuresdf)

,feature,class_label
0,"[-314.87665, 166.32927, 4.413409, 16.052296, 0...",dog_bark
1,"[-181.81837, 137.67917, -77.676834, -45.46656,...",dog_bark
2,"[-323.49872, 140.18324, -11.821219, 14.91496, ...",dog_bark
3,"[-585.21686, 93.08137, -30.279652, 6.409308, -...",dog_bark
4,"[-224.30125, 173.66426, -23.907955, 18.138966,...",dog_bark
...,...,...
1491,"[-255.47444, 148.7255, -50.32463, 5.921347, -1...",siren
1492,"[-214.6939, 82.65424, -34.92253, 1.847175, -28...",siren
1493,"[-271.99802, 143.69476, -77.24437, -0.6722971,...",siren
1494,"[-422.82904, 160.23573, -64.7399, -24.466932, ...",siren


In [5]:
# Use sklearn.preprocessing.LabelEncoder to encode the categorical text data into model-understandable numerical data

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

display(y)

#This part will convert the categories into their respective numerical value
le = LabelEncoder()
# Fit transform receives categories and assigns numerical value to them. to_categorical converts to binary matrix
yy = to_categorical(le.fit_transform(y))
unique_rows = np.unique(yy, axis=0)
display(unique_rows)

Using TensorFlow backend.


array(['dog_bark', 'dog_bark', 'dog_bark', ..., 'siren', 'siren', 'siren'],
      dtype='<U8')

array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [6]:
#print(yy[0])

#for i in yy:
 #   print(i)
  #  if i == [1., 0., 0., 0]
   #     print("woo")
    #Xdog
display(featuresdf.feature)

0       [-314.87665, 166.32927, 4.413409, 16.052296, 0...
1       [-181.81837, 137.67917, -77.676834, -45.46656,...
2       [-323.49872, 140.18324, -11.821219, 14.91496, ...
3       [-585.21686, 93.08137, -30.279652, 6.409308, -...
4       [-224.30125, 173.66426, -23.907955, 18.138966,...
                              ...                        
1491    [-255.47444, 148.7255, -50.32463, 5.921347, -1...
1492    [-214.6939, 82.65424, -34.92253, 1.847175, -28...
1493    [-271.99802, 143.69476, -77.24437, -0.6722971,...
1494    [-422.82904, 160.23573, -64.7399, -24.466932, ...
1495    [-609.3688, 176.50005, -71.21106, -10.201033, ...
Name: feature, Length: 1496, dtype: object

In [7]:
# Split the Data 
# Need to split for training vs. testing (80% vs. 20%)

from sklearn.model_selection import train_test_split

# X is feature, Y is labels
# 42 is the seed to generating random numbers - starting position, integer required to ensure training and testing are consistent
#x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.1, random_state=42)
#print(x_train, x_test, y_train, y_test)


In [8]:
# Store data into next notebook
%store x_train
%store x_test
%store y_test
%store y_train
%store yy
%store le

Stored 'x_train' (ndarray)
Stored 'x_test' (ndarray)
Stored 'y_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'yy' (ndarray)
Stored 'le' (LabelEncoder)
